# Generatywne sieci

Rekurencyjne Sieci Neuronowe (RNN) oraz ich warianty z komórkami bramkowymi, takie jak Komórki Długiej Krótkoterminowej Pamięci (LSTM) i Bramkowe Jednostki Rekurencyjne (GRU), dostarczają mechanizmu do modelowania języka, czyli potrafią uczyć się kolejności słów i przewidywać następne słowo w sekwencji. Dzięki temu możemy używać RNN do **zadań generatywnych**, takich jak zwykłe generowanie tekstu, tłumaczenie maszynowe, a nawet opisywanie obrazów.

W architekturze RNN, którą omawialiśmy w poprzedniej jednostce, każda jednostka RNN generowała kolejny ukryty stan jako wynik. Jednak możemy również dodać kolejny wynik do każdej rekurencyjnej jednostki, co pozwoli nam generować **sekwencję** (równą długości oryginalnej sekwencji). Co więcej, możemy używać jednostek RNN, które nie przyjmują wejścia na każdym kroku, a jedynie korzystają z początkowego wektora stanu, aby następnie generować sekwencję wyników.

W tym notatniku skupimy się na prostych modelach generatywnych, które pomagają nam generować tekst. Dla uproszczenia zbudujmy **sieć na poziomie znaków**, która generuje tekst litera po literze. Podczas treningu musimy wziąć jakiś korpus tekstu i podzielić go na sekwencje liter.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Tworzenie słownika znaków

Aby zbudować sieć generatywną na poziomie znaków, musimy podzielić tekst na pojedyncze znaki zamiast słów. Warstwa `TextVectorization`, której używaliśmy wcześniej, nie pozwala na to, więc mamy dwie opcje:

* Ręczne wczytanie tekstu i tokenizacja "na piechotę", jak w [tym oficjalnym przykładzie Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Użycie klasy `Tokenizer` do tokenizacji na poziomie znaków.

Wybierzemy drugą opcję. `Tokenizer` może być również używany do tokenizacji na poziomie słów, więc przejście z tokenizacji znakowej na słowną powinno być dość proste.

Aby przeprowadzić tokenizację na poziomie znaków, musimy przekazać parametr `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Chcemy również użyć jednego specjalnego tokenu, aby oznaczyć **koniec sekwencji**, który nazwiemy `<eos>`. Dodajmy go ręcznie do słownika:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

Teraz, aby zakodować tekst w sekwencje liczb, możemy użyć:


In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Trenowanie generatywnej RNN do tworzenia tytułów

Sposób, w jaki będziemy trenować RNN do generowania tytułów wiadomości, wygląda następująco. Na każdym kroku weźmiemy jeden tytuł, który zostanie wprowadzony do RNN, a dla każdego znaku wejściowego poprosimy sieć o wygenerowanie kolejnego znaku wyjściowego:

![Obraz przedstawiający przykład generowania słowa 'HELLO' przez RNN.](../../../../../lessons/5-NLP/17-GenerativeNetworks/images/rnn-generate.png)

Dla ostatniego znaku naszej sekwencji poprosimy sieć o wygenerowanie tokenu `<eos>`.

Główna różnica w generatywnej RNN, którą tutaj wykorzystujemy, polega na tym, że będziemy pobierać wyjście z każdego kroku RNN, a nie tylko z ostatniej komórki. Można to osiągnąć, ustawiając parametr `return_sequences` dla komórki RNN.

Tak więc, podczas treningu, wejściem do sieci będzie sekwencja zakodowanych znaków o określonej długości, a wyjściem będzie sekwencja o tej samej długości, ale przesunięta o jeden element i zakończona tokenem `<eos>`. Minibatch będzie składać się z kilku takich sekwencji, a do wyrównania wszystkich sekwencji będziemy musieli użyć **paddingu**.

Stwórzmy funkcje, które przekształcą dla nas zbiór danych. Ponieważ chcemy uzupełniać sekwencje na poziomie minibatch, najpierw podzielimy zbiór danych, wywołując `.batch()`, a następnie użyjemy `map`, aby dokonać transformacji. Funkcja transformacji będzie więc przyjmować cały minibatch jako parametr:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Kilka ważnych rzeczy, które robimy tutaj:
* Najpierw wyciągamy rzeczywisty tekst z tensora tekstowego
* `text_to_sequences` konwertuje listę ciągów znaków na listę tensora liczb całkowitych
* `pad_sequences` uzupełnia te tensory do ich maksymalnej długości
* Na końcu kodujemy wszystkie znaki w formacie one-hot, a także wykonujemy przesunięcie i dodajemy `<eos>`. Wkrótce zobaczymy, dlaczego potrzebujemy znaków zakodowanych w formacie one-hot.

Jednak ta funkcja jest **Pythonowa**, czyli nie może być automatycznie przetłumaczona na graf obliczeniowy Tensorflow. Otrzymamy błędy, jeśli spróbujemy użyć tej funkcji bezpośrednio w funkcji `Dataset.map`. Musimy otoczyć to Pythonowe wywołanie, używając wrappera `py_function`:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Uwaga**: Rozróżnienie między funkcjami transformacji w stylu Pythonowym a funkcjami Tensorflow może wydawać się zbyt skomplikowane i możesz się zastanawiać, dlaczego nie przekształcamy zbioru danych za pomocą standardowych funkcji Pythona przed przekazaniem go do `fit`. Chociaż jest to jak najbardziej możliwe, użycie `Dataset.map` ma ogromną zaletę, ponieważ pipeline przekształcania danych jest wykonywany za pomocą grafu obliczeniowego Tensorflow, co pozwala wykorzystać obliczenia na GPU i minimalizuje konieczność przesyłania danych między CPU a GPU.

Teraz możemy zbudować naszą sieć generatora i rozpocząć trening. Może być oparta na dowolnej komórce rekurencyjnej, którą omówiliśmy w poprzedniej jednostce (prosta, LSTM lub GRU). W naszym przykładzie użyjemy LSTM.

Ponieważ sieć przyjmuje znaki jako dane wejściowe, a rozmiar słownika jest dość mały, nie potrzebujemy warstwy osadzania (embedding). Dane wejściowe zakodowane w formacie one-hot mogą być bezpośrednio przekazane do komórki LSTM. Warstwa wyjściowa będzie klasyfikatorem `Dense`, który przekształci wynik LSTM w liczby tokenów zakodowanych w formacie one-hot.

Dodatkowo, ponieważ mamy do czynienia z sekwencjami o zmiennej długości, możemy użyć warstwy `Masking`, aby utworzyć maskę ignorującą wypełnione (padded) części ciągu. Nie jest to ściśle konieczne, ponieważ nie interesuje nas szczególnie wszystko, co znajduje się poza tokenem `<eos>`, ale użyjemy jej, aby zdobyć doświadczenie z tego typu warstwą. `input_shape` będzie miało wartość `(None, vocab_size)`, gdzie `None` wskazuje na sekwencję o zmiennej długości, a kształt wyjściowy to również `(None, vocab_size)`, jak można zobaczyć w `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Generowanie wyników

Teraz, gdy model został wytrenowany, chcemy go użyć do generowania wyników. Przede wszystkim potrzebujemy sposobu na dekodowanie tekstu reprezentowanego przez sekwencję numerów tokenów. W tym celu moglibyśmy użyć funkcji `tokenizer.sequences_to_texts`; jednak nie działa ona dobrze z tokenizacją na poziomie znaków. Dlatego weźmiemy słownik tokenów z tokenizera (nazywany `word_index`), zbudujemy odwrotną mapę i napiszemy własną funkcję dekodującą:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Teraz rozpoczniemy generowanie. Zaczniemy od ciągu znaków `start`, zakodujemy go w sekwencję `inp`, a następnie na każdym kroku wywołamy naszą sieć, aby przewidzieć kolejny znak.

Wyjście sieci `out` to wektor z `vocab_size` elementami, reprezentujący prawdopodobieństwa każdego tokenu. Możemy znaleźć najbardziej prawdopodobny numer tokenu, używając `argmax`. Następnie dodajemy ten znak do wygenerowanej listy tokenów i kontynuujemy generowanie. Ten proces generowania jednego znaku powtarza się `size` razy, aby wygenerować wymaganą liczbę znaków, a proces kończymy wcześniej, gdy napotkamy `eos_token`.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Próbkowanie wyników podczas treningu

Ponieważ nie mamy żadnych użytecznych metryk, takich jak *dokładność*, jedynym sposobem, aby zobaczyć, że nasz model się poprawia, jest **próbkowanie** generowanego ciągu podczas treningu. Aby to zrobić, użyjemy **callbacków**, czyli funkcji, które możemy przekazać do funkcji `fit`, a które będą wywoływane okresowo podczas treningu.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Ten przykład generuje już całkiem dobry tekst, ale można go jeszcze ulepszyć na kilka sposobów:
* **Więcej tekstu**. Użyliśmy tylko tytułów dla naszego zadania, ale warto poeksperymentować z pełnym tekstem. Pamiętaj, że RNN-y nie radzą sobie najlepiej z obsługą długich sekwencji, więc sensowne jest albo podzielenie ich na krótsze zdania, albo zawsze trenowanie na stałej długości sekwencji o jakiejś z góry określonej wartości `num_chars` (np. 256). Możesz spróbować zmienić powyższy przykład na taką architekturę, korzystając z [oficjalnego samouczka Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/) jako inspiracji.
* **Wielowarstwowy LSTM**. Warto spróbować użyć 2 lub 3 warstw komórek LSTM. Jak wspomnieliśmy w poprzedniej jednostce, każda warstwa LSTM wyodrębnia określone wzorce z tekstu, a w przypadku generatora na poziomie znaków możemy oczekiwać, że niższy poziom LSTM będzie odpowiedzialny za wyodrębnianie sylab, a wyższe poziomy - za słowa i kombinacje słów. Można to łatwo zaimplementować, przekazując parametr liczby warstw do konstruktora LSTM.
* Możesz również poeksperymentować z **jednostkami GRU** i sprawdzić, które działają lepiej, oraz z **różnymi rozmiarami warstw ukrytych**. Zbyt duża warstwa ukryta może prowadzić do nadmiernego dopasowania (np. sieć nauczy się dokładnego tekstu), a mniejszy rozmiar może nie dawać dobrych wyników.


## Generowanie miękkiego tekstu i temperatura

W poprzedniej definicji `generate` zawsze wybieraliśmy znak o najwyższym prawdopodobieństwie jako kolejny znak w generowanym tekście. Skutkowało to tym, że tekst często "cyklicznie" powtarzał te same sekwencje znaków, jak w tym przykładzie:
```
today of the second the company and a second the company ...
```

Jednakże, jeśli spojrzymy na rozkład prawdopodobieństwa dla kolejnego znaku, może się okazać, że różnica między kilkoma najwyższymi prawdopodobieństwami nie jest duża, np. jeden znak może mieć prawdopodobieństwo 0.2, a inny 0.19, itd. Na przykład, gdy szukamy kolejnego znaku w sekwencji '*play*', kolejnym znakiem może być równie dobrze spacja, jak i **e** (jak w słowie *player*).

Prowadzi nas to do wniosku, że nie zawsze jest "sprawiedliwe" wybieranie znaku o najwyższym prawdopodobieństwie, ponieważ wybór drugiego najwyższego może również prowadzić do sensownego tekstu. Bardziej rozsądne jest **losowanie** znaków z rozkładu prawdopodobieństwa podanego przez wynik sieci.

Takie losowanie można przeprowadzić za pomocą funkcji `np.multinomial`, która implementuje tzw. **rozkład wielomianowy**. Funkcja, która implementuje to **miękkie** generowanie tekstu, jest zdefiniowana poniżej:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Wprowadziliśmy jeszcze jeden parametr o nazwie **temperatura**, który służy do wskazania, jak bardzo powinniśmy trzymać się najwyższego prawdopodobieństwa. Jeśli temperatura wynosi 1.0, przeprowadzamy uczciwe próbkowanie multinomialne, a gdy temperatura dąży do nieskończoności - wszystkie prawdopodobieństwa stają się równe i losowo wybieramy następny znak. Na poniższym przykładzie możemy zaobserwować, że tekst staje się bezsensowny, gdy zbytnio zwiększymy temperaturę, a przypomina "cyklicznie" generowany tekst, gdy zbliża się do 0.



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby zapewnić poprawność tłumaczenia, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za źródło autorytatywne. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
